In [1]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import os

In [3]:
os.chdir(r'/home/developer/projects/spark-course-python/spark_course_python/projects/spark/class/2-DataFrame')
spark = SparkSession.builder.getOrCreate()
# df_people = spark.read.csv('people.txt',header = True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/17 16:06:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
simpleData = (("James", "Sales", 3000), ("Michael", "Sales", 4600),\
              ("James", "Sales", 3000),("Scott", "Finance", 3300), \
              ("Jen", "Finance", 3900), ("Jeff", "Marketing", 3000),\
              ("Kumar", "Marketing", 2000), ("Saif", "Sales", 4100))


columns = ["employee_name", "department", "salary"]
df = spark.createDataFrame(data=simpleData, schema=columns)
# df.printSchema()
#df.show(truncate=False)

In [6]:
#row_number
windowSpec = Window.partitionBy("department").orderBy("salary")
df.withColumn('rn',F.row_number().over(windowSpec)).show(10)

+-------------+----------+------+---+
|employee_name|department|salary| rn|
+-------------+----------+------+---+
|        Scott|   Finance|  3300|  1|
|          Jen|   Finance|  3900|  2|
|        Kumar| Marketing|  2000|  1|
|         Jeff| Marketing|  3000|  2|
|        James|     Sales|  3000|  1|
|        James|     Sales|  3000|  2|
|         Saif|     Sales|  4100|  3|
|      Michael|     Sales|  4600|  4|
+-------------+----------+------+---+



In [49]:
#agg_functions
windowSpecAgg  = Window.partitionBy('department')
df.withColumn('max',max(F.col('salary')).over(windowSpecAgg)).show(10)

+-------------+----------+------+----+
|employee_name|department|salary| max|
+-------------+----------+------+----+
|        Scott|   Finance|  3300|3900|
|          Jen|   Finance|  3900|3900|
|         Jeff| Marketing|  3000|3000|
|        Kumar| Marketing|  2000|3000|
|        James|     Sales|  3000|4600|
|      Michael|     Sales|  4600|4600|
|        James|     Sales|  3000|4600|
|         Saif|     Sales|  4100|4600|
+-------------+----------+------+----+



In [54]:
#dense_rank
windowSpec = Window.partitionBy("department").orderBy("salary")
df.withColumn("dense_rank",F.dense_rank().over(windowSpec)) .show()

+-------------+----------+------+----------+
|employee_name|department|salary|dense_rank|
+-------------+----------+------+----------+
|        Scott|   Finance|  3300|         1|
|          Jen|   Finance|  3900|         2|
|        Kumar| Marketing|  2000|         1|
|         Jeff| Marketing|  3000|         2|
|        James|     Sales|  3000|         1|
|        James|     Sales|  3000|         1|
|         Saif|     Sales|  4100|         2|
|      Michael|     Sales|  4600|         3|
+-------------+----------+------+----------+



In [61]:
#ntile
#מחלק כל קטגוריה לקבוצות ע"פ מה שהוגדר
windowSpec = Window.partitionBy("department").orderBy("salary")
df.withColumn("ntile",F.ntile(2).over(windowSpec)).show()

+-------------+----------+------+-----+
|employee_name|department|salary|ntile|
+-------------+----------+------+-----+
|        Scott|   Finance|  3300|    1|
|          Jen|   Finance|  3900|    2|
|         Jeff| Marketing|  3000|    1|
|        Kumar| Marketing|  2000|    2|
|        James|     Sales|  3000|    1|
|      Michael|     Sales|  4600|    1|
|        James|     Sales|  3000|    2|
|         Saif|     Sales|  4100|    2|
+-------------+----------+------+-----+



In [71]:
#percent_rank
#מחשב את האחוזון של כל שורה
windowSpec = Window.partitionBy("department").orderBy("salary")
df.withColumn("percent_rank",F.percent_rank().over(windowSpec)).show()

+-------------+----------+------+------------------+
|employee_name|department|salary|      percent_rank|
+-------------+----------+------+------------------+
|        Scott|   Finance|  3300|               0.0|
|          Jen|   Finance|  3900|               1.0|
|        Kumar| Marketing|  2000|               0.0|
|         Jeff| Marketing|  3000|               1.0|
|        James|     Sales|  3000|               0.0|
|        James|     Sales|  3000|               0.0|
|         Saif|     Sales|  4100|0.6666666666666666|
|      Michael|     Sales|  4600|               1.0|
+-------------+----------+------+------------------+



In [72]:
#rank
#בכל קטגוריה סופר את מספר האיברים הדומים, אך אם יש שני ערכים אותו דבר הוא נותן להם אותו מספר וקובץ מספר קדימה.
windowSpec = Window.partitionBy("department").orderBy("salary")
df.withColumn("rank",F.rank().over(windowSpec)).show()

+-------------+----------+------+----+
|employee_name|department|salary|rank|
+-------------+----------+------+----+
|        Scott|   Finance|  3300|   1|
|          Jen|   Finance|  3900|   2|
|        Kumar| Marketing|  2000|   1|
|         Jeff| Marketing|  3000|   2|
|        James|     Sales|  3000|   1|
|        James|     Sales|  3000|   1|
|         Saif|     Sales|  4100|   3|
|      Michael|     Sales|  4600|   4|
+-------------+----------+------+----+



In [74]:
#cume_dist
#מחשבת את התפלגות הערכים בתוך הקטגוריה  אך כן מתחשבת בערכים שווים בניגוד ל
#percent_rank
windowSpec = Window.partitionBy("department").orderBy("salary")
df.withColumn("cume_dist",F.cume_dist().over(windowSpec)).show()

+-------------+----------+------+---------+
|employee_name|department|salary|cume_dist|
+-------------+----------+------+---------+
|        Scott|   Finance|  3300|      0.5|
|          Jen|   Finance|  3900|      1.0|
|        Kumar| Marketing|  2000|      0.5|
|         Jeff| Marketing|  3000|      1.0|
|        James|     Sales|  3000|      0.5|
|        James|     Sales|  3000|      0.5|
|         Saif|     Sales|  4100|     0.75|
|      Michael|     Sales|  4600|      1.0|
+-------------+----------+------+---------+



In [82]:
#lag
#מביא את הערך של מספר שורות אחורה שמוגדרות בתוך קטגוריה מסוימת
windowSpec = Window.partitionBy("department").orderBy("salary")
df.withColumn("lag",F.lag("salary",1).over(windowSpec)).show()


+-------------+----------+------+----+
|employee_name|department|salary| lag|
+-------------+----------+------+----+
|        Scott|   Finance|  3300|null|
|          Jen|   Finance|  3900|3300|
|        Kumar| Marketing|  2000|null|
|         Jeff| Marketing|  3000|2000|
|        James|     Sales|  3000|null|
|        James|     Sales|  3000|3000|
|         Saif|     Sales|  4100|3000|
|      Michael|     Sales|  4600|4100|
+-------------+----------+------+----+



In [85]:
#lead
#מביא את הערך של מספר שורות קדימה שמוגדרות בתוך קטגוריה מסוימת
windowSpec = Window.partitionBy("department").orderBy("salary")
df.withColumn("lead",F.lead("salary",1).over(windowSpec)).show()

+-------------+----------+------+----+
|employee_name|department|salary|lead|
+-------------+----------+------+----+
|        Scott|   Finance|  3300|3900|
|          Jen|   Finance|  3900|null|
|        Kumar| Marketing|  2000|3000|
|         Jeff| Marketing|  3000|null|
|        James|     Sales|  3000|3000|
|        James|     Sales|  3000|4100|
|         Saif|     Sales|  4100|4600|
|      Michael|     Sales|  4600|null|
+-------------+----------+------+----+

